<H1><center>Обработка естественного языка</center></H1>

<H2>Введение</H2>

В данной лабораторной работе мы рассмотрим одну из моделей для обработки естественного языка - rubert-tiny  
Rubert-tiny является уменьшенной версией BERT, ориентированной всего на 2 языка: русский и английский (отсюда и название)

В качестве примера вам будет дан код обучения модели на датасете CoLA (Corpus of Linguistic Acceptability)  
Входными данными в этом датасете являются различные предложения на английском языке, а предсказывается правильность их построения

Затем вы должны будете самостоятельно обучить эту же (можно другую) модель на датасете с отзывами к фильмам  
Обученная модель должна определять тип отзыва - положительный или отрицательный

<H4>Цели</H4>

- Изучить принципы обработки естественного языка
- Обучить нейронную сеть rubert-tiny определять тип отзыва на фильм (положительный/отрицательный)

<H4>Задачи</H4>

<H4>Обозначения</H4>

\[1] ⭐ - Задание (звездочки - баллы, число - номер задания)  
\[1] 💫 - Конец задания

<H2>Обучение rubert-tiny на датасете CoLA</H2>

<H3>1. Подготовка данных</H3>

Как обычно, наша первая задача - подготовить данные для обучения  
Считаем таблицу "сырых" текстов и посмотрим, что в ней лежит

In [1]:
import pandas as pd

# Загружаем dataset в pandas dataframe.
df = pd.read_csv("./data/cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

sentences = df['sentence'].values
labels = df['label'].values

In [2]:
# Выводим число тренировочных предложений.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Выводим случайные 10 рядов из таблички.
print(df.sample(10))

# Выводим 5 грамматически неверных предложений.
print(df.loc[df.label == 0].sample(5)[['sentence', 'label']])

Number of training sentences: 8,551

     sentence_source  label label_notes  \
87              gj04      1         NaN   
1483            r-67      1         NaN   
4610            ks08      1         NaN   
6279            c_13      1         NaN   
4559            ks08      0           *   
3131            l-93      1         NaN   
6544            g_81      0           *   
6122            c_13      1         NaN   
6042            c_13      0           *   
6412            d_98      1         NaN   

                                               sentence  
87                             The tiger bled to death.  
1483       A review came out yesterday of this article.  
4610             John has chosen Bill for the position.  
6279         To improve myself is a goal for next year.  
4559          It has rains every day for the last week.  
3131                                      Paul exhaled.  
6544  The table, I put Kim on which supported the book.  
6122                     

<br>
<H4>1.1. Токенизация</H4>

Теперь входные тексты нужно привести к формату, который может прочитать языковая модель  
В этом нам поможет другая модель - токенизатор  

In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('./models/rubert-tiny', do_lower_case=True)

<br>Сначала токенизатор разбивает текст на составные части (токены), которые есть в его словаре  
Токеном может быть слово, часть слова или символ, а также одно из специальных обозначений, используемых при обучении  

In [4]:
sentence = sentences[0]
print('Original:', sentence)                                 # Вывести оригинальное предложение.
print('Tokenized: ', tokenizer.tokenize(sentence))           # Вывести предложение, разбитое на отдельные токены из словаря.

Original: Our friends won't buy this analysis, let alone the next one we propose.
Tokenized:  ['our', 'friends', 'won', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.']


<br>Словарь нашего токенизатора находится в файле ./models/rubert-tiny/vocab.txt

<center>[1] ⭐</center>

Откройте словарь токенизатора и найдите в нем:
- русское слово
- часть русского слова
- специальный токен

⭐ Объясните значение специального токена  

P.S. Все 3 токена должны отличаться от токенов ваших соседей

журнала
дзе
[CLS]

<center>[1] 💫</center>

Токены затем конвертируются в их идентификаторы, которые уже можно подавать на вход языковой модели

In [5]:
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence)))       # Вывести предложение, разбитое на номера токенов в словаре.

Token IDs:  [4589, 6597, 1427, 11, 87, 15085, 881, 5356, 16, 2703, 7421, 531, 2624, 835, 1798, 9529, 18]


<br>То же самое можно сделать в один шаг, для всего датасета и с добавлением специальных токенов

In [6]:
encoded_dict = tokenizer.batch_encode_plus(
    list(sentences),                           # Текст для токенизации.
    add_special_tokens=True,                   # Добавляем '[CLS]' и '[SEP]'
    max_length=64,                             # Дополняем [PAD] или обрезаем текст до 64 токенов.
    padding='max_length',
    truncation=True,
    return_attention_mask=True,                # Возвращаем также attn. masks.
    return_tensors='pt',                       # Возвращаем в виде тензоров pytorch.
)

print(f'Token IDs:\n{encoded_dict["input_ids"]}')
print(f'Attention masks:\n{encoded_dict["attention_mask"]}')

Token IDs:
tensor([[   2, 4589, 6597,  ...,    0,    0,    0],
        [   2,  835, 1052,  ...,    0,    0,    0],
        [   2,  835, 1052,  ...,    0,    0,    0],
        ...,
        [   2,  683,  550,  ...,    0,    0,    0],
        [   2,   76,  768,  ...,    0,    0,    0],
        [   2, 2376,  813,  ...,    0,    0,    0]])
Attention masks:
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


<center>[2] ⭐</center>

Закодируйте свой отзыв на фильм с помощью токенизатора  
Используйте <code>encode_plus</code> вместо <code>batch_encode_plus</code>, чтобы передать один текст, а не список

Выведите токены (не их id) с помощью <code>tokenize</code>

In [7]:
rew = "Замечательный криминальный фильм-комедия. По своему выдающийся, местами запоминающийся, этот фильм не романтизирует образ преступников, наоборот обличает, хоть и в ироничной форме."

encoded_dict_rew = tokenizer.encode_plus(
    list(rew),                           # Текст для токенизации.
    add_special_tokens=True,                   # Добавляем '[CLS]' и '[SEP]'
    max_length=64,                             # Дополняем [PAD] или обрезаем текст до 64 токенов.
    padding='max_length',
    truncation=True,
    return_attention_mask=True,                # Возвращаем также attn. masks.
    return_tensors='pt',                       # Возвращаем в виде тензоров pytorch.
)

print(tokenizer.tokenize(rew))
print(f'Token IDs:\n{encoded_dict["input_ids"]}')
print(f'Attention masks:\n{encoded_dict["attention_mask"]}')

['за', '##ме', '##чат', '##ель', '##ны', '##и', 'к', '##рим', '##инал', '##ьны', '##и', 'фильм', '-', 'коме', '##дия', '.', 'по', 'своему', 'вы', '##да', '##ю', '##щи', '##ися', ',', 'места', '##ми', 'за', '##пом', '##ина', '##ю', '##щи', '##ися', ',', 'этот', 'фильм', 'не', 'роман', '##ти', '##зи', '##рует', 'образ', 'пре', '##ступ', '##ников', ',', 'на', '##об', '##орот', 'обл', '##ича', '##ет', ',', 'хот', '##ь', 'и', 'в', 'и', '##рони', '##чно', '##и', 'форме', '.']
Token IDs:
tensor([[   2, 4589, 6597,  ...,    0,    0,    0],
        [   2,  835, 1052,  ...,    0,    0,    0],
        [   2,  835, 1052,  ...,    0,    0,    0],
        ...,
        [   2,  683,  550,  ...,    0,    0,    0],
        [   2,   76,  768,  ...,    0,    0,    0],
        [   2, 2376,  813,  ...,    0,    0,    0]])
Attention masks:
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ...

<center>[2] 💫</center>

<H4>1.2. Загрузчики данных</H4>

Итерироваться по данным для обучения и валидации будем с помощью уже известных нам объектов <code>DataLoader</code>

Для начала соберем наши данные в класс <code>TensorDataset</code>  
Нас интересуют закодированные токены, маски внимания (чтобы игнорировать пустые токены) и выходные значения

In [8]:
import torch
from torch.utils.data import TensorDataset

input_ids = encoded_dict['input_ids']
attention_masks = encoded_dict['attention_mask']
labels = torch.tensor(labels)

dataset = TensorDataset(input_ids, attention_masks, labels)

<br>Теперь поделим данные на тренировочные и валидационные в соотношении 90/10

In [9]:
from torch.utils.data import random_split

# Считаем число данных для тренировки и для валидации.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Разбиваем датасет с учетом посчитанного количества.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

7,695 training samples
  856 validation samples


<br>Создадим свой загрузчик для тренировочного набора

In [10]:
from torch.utils.data import DataLoader

# Задаем размер батча для тренировки.
# Авторы BERT предлагают ставить его 16 или 32. 
batch_size = 32

train_dataloader = DataLoader(
        train_dataset,
        shuffle=True,              # берем данные в случайном порядке
        batch_size = batch_size
)

<center>[3] ⭐</center>

А теперь создайте загрузчик для валидационного набора (назовите переменную <code>validation_dataloader</code>, она понадобится дальше)  
Данные должны считываться по порядку, без перемешивания

In [11]:
batch_size = 32

validation_dataloader = DataLoader(
        val_dataset,
        shuffle = False,
        batch_size = batch_size
   
)

<center>[3] 💫</center>

<br>
<H3>2. Подготовка модели</H3>

Сама модель лежит там же, где и токенизатор - в ./models/rubert-tiny  
Мы используем предобученную модель и дообучаем ее на своей задаче

In [12]:
from transformers import BertForSequenceClassification

# Загружаем BertForSequenceClassification. Это предобученная модель BERT с одиночным полносвязным слоем для классификации
model = BertForSequenceClassification.from_pretrained(
    "./models/rubert-tiny",                                 # Используем rubert-tiny.
    num_labels = 2,                                         # Количество выходных слоёв – 2 для бинарной классификации.
    output_attentions = False,                              # Будет ли модель возвращать веса для attention-слоёв. В нашем случае нет.
    output_hidden_states = False,                           # Будет ли модель возвращать состояние всех скрытых слоёв. В нашем случае нет.
)

# При наличии GPU переносим модель туда.
if torch.cuda.is_available():
    model.cuda()

<br>Посмотрим структуру нашей модели

In [13]:
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 57 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (29564, 312)
bert.embeddings.position_embeddings.weight                (512, 312)
bert.embeddings.token_type_embeddings.weight                (2, 312)
bert.embeddings.LayerNorm.weight                              (312,)
bert.embeddings.LayerNorm.bias                                (312,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (312, 312)
bert.encoder.layer.0.attention.self.query.bias                (312,)
bert.encoder.layer.0.attention.self.key.weight            (312, 312)
bert.encoder.layer.0.attention.self.key.bias                  (312,)
bert.encoder.layer.0.attention.self.value.weight          (312, 312)
bert.encoder.layer.0.attention.self.value.bias                (312,)
bert.encoder.layer.0.attention.output.dense.weight        (312, 312)
bert.encoder.layer.0.attention.output.dense.bias              (3

<br>  

<i>\- Все понятно?  
\- Нет!  
\- Идем дальше</i>

Помимо самой модели, нам понадобится оптимизатор и планировщик коэффициента скорости обучения  

In [15]:
from transformers import AdamW, get_linear_schedule_with_warmup

# Берем оптимизатор AdamW
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

# Количество эпох для тренировки. Авторы BERT рекомендуют от 2 до 4.
# Мы выбираем 4, но увидим позже, что это приводит к оверфиту на тренировочные данные.
epochs = 4

# Общее число шагов тренировки равно [количество батчей] x [число эпох].
total_steps = len(train_dataloader) * epochs

# Создаем планировщик learning rate (LR). LR будет плавно уменьшаться в процессе тренировки.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

<br>
<H3>3. Обучение модели</H3>

Определим, на каком устройстве нам нужны входные данные для модели: GPU или CPU

In [16]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

<br>Добавим пару вспомогательных функций

In [17]:
import numpy as np

# Функция для расчёта точности. Сравниваются предсказания и реальная разметка к данным
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


import time
import datetime
import random 

# Функция переводит время в секундах в формат hh:mm:ss
def format_time(elapsed):
    # Округляем до ближайшей секунды.
    elapsed_rounded = int(round((elapsed)))

    # Форматируем как hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

<br>Теперь функция обучения (для одной эпохи)

In [18]:
def train_step(
        device,               # Устройство для вычислений
        model,                # Модель (в нашем случае rubert-tiny)
        train_dataloader,     # Загрузчик тренировочных данных
        optimizer,            # Оптимизатор (в нашем случае AdamW)
        scheduler             # Планировщик коэффициента скорости обучения
):
    t0 = time.time()
    total_train_loss = 0

    model.train()     # Переводим модель в режим тренировки

    # Итерируемся по батчам из тренировочных данных
    for step, batch in enumerate(train_dataloader):
        
        # Выводим прогресс каждый 10й батч для удобства
        if step % 10 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Извлекаем все компоненты из полученного батча
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        
        # Очищаем все ранее посчитанные градиенты (это важно)
        model.zero_grad()
        
        # Выполняем прямой проход по данным
        out = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = out.loss
        logits = out.logits
        
        # Накапливаем тренировочную функцию потерь по всем батчам
        total_train_loss += loss.item()
        
        # Выполняем обратное распространение ошибки, чтобы посчитать градиенты.
        loss.backward()
        
        # Ограничиваем максимальный размер градиента до 1.0. Это позволяет избежать проблемы "exploding gradients".
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # Обновляем параметры модели, используя рассчитанные градиенты с помощью выбранного оптимизатора и текущего learning rate.
        optimizer.step()
        
        # Обновляем learning rate.
        scheduler.step()

    # Считаем среднее значение функции потерь по всем батчам.
    avg_train_loss = total_train_loss / len(train_dataloader)
    
    # Сохраняем время тренировки одной эпохи.
    training_time = format_time(time.time() - t0)
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoсh took: {:}".format(training_time))
    return avg_train_loss, training_time

<br>И функция валидации 

In [19]:
def validation_step(
        device,                  # Устройство для вычислений
        model,                   # Модель (в нашем случае rubert-tiny)
        validation_dataloader    # Планировщик коэффициента скорости обучения
):
    print("Running Validation...")
    t0 = time.time()
    
    # Переводим модель в режим evaluation: некоторые слои, например dropout, ведут себя по-другому.
    model.eval()

    # Переменные для подсчёта функции потерь и точности
    total_eval_accuracy = 0
    total_eval_loss = 0
    
    # Прогоняем все данные из валидации
    for step, batch in enumerate(validation_dataloader):

        # Выводим прогресс каждый 10й батч для удобства
        if step % 10 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(validation_dataloader), elapsed))

        # Извлекаем все компоненты из полученного батча.
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)

        # Говорим pytorch, что нам не нужен вычислительный граф для подсчёта градиентов (всё будет работать намного быстрее)
        with torch.no_grad():
            # Прямой проход по нейронной сети и получение выходных значений.
            out = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss = out.loss
            logits = out.logits

        # Накапливаем значение функции потерь для валидации.
        total_eval_loss += loss.item()

        # Переносим значения с GPU на CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Считаем точность для отдельного батча с текстами и накапливаем значения.
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    # Выводим точность для всех валидационных данных.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Считаем среднюю функцию потерь для всех батчей.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Измеряем, как долго считалась валидация.
    validation_time = format_time(time.time() - t0)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    return avg_val_loss, avg_val_accuracy, validation_time

<center>[4] ⭐⭐</center>

Используя функции <code>train_step</code> и <code>validation_step</code>, реализуйте цикл обучения модели  
Количество итераций хранится в переменной <code>epochs</code>  
Для удобства на каждой эпохе сделайте вывод ее номера

<i>Прим.: ваша главная задача здесь - внимательно посмотреть на аргументы обеих функций и понять, какие значения им передать  
Все эти значения уже хранятся в переменных, никаких дополнительных вычислений не нужно</i>

In [20]:
epochs = 0
for i in range(4):
    epochs = 1
    i
    train_step(device, model, train_dataloader, optimizer, scheduler)
    validation_step(device, model, validation_dataloader)
    print(epochs)

  Batch    10  of    241.    Elapsed: 0:00:11.
  Batch    20  of    241.    Elapsed: 0:00:19.
  Batch    30  of    241.    Elapsed: 0:00:28.
  Batch    40  of    241.    Elapsed: 0:00:36.
  Batch    50  of    241.    Elapsed: 0:00:46.
  Batch    60  of    241.    Elapsed: 0:00:55.
  Batch    70  of    241.    Elapsed: 0:01:05.
  Batch    80  of    241.    Elapsed: 0:01:16.
  Batch    90  of    241.    Elapsed: 0:01:26.
  Batch   100  of    241.    Elapsed: 0:01:36.
  Batch   110  of    241.    Elapsed: 0:01:46.
  Batch   120  of    241.    Elapsed: 0:01:57.
  Batch   130  of    241.    Elapsed: 0:02:07.
  Batch   140  of    241.    Elapsed: 0:02:17.
  Batch   150  of    241.    Elapsed: 0:02:27.
  Batch   160  of    241.    Elapsed: 0:02:38.
  Batch   170  of    241.    Elapsed: 0:02:48.
  Batch   180  of    241.    Elapsed: 0:02:57.
  Batch   190  of    241.    Elapsed: 0:03:06.
  Batch   200  of    241.    Elapsed: 0:03:15.
  Batch   210  of    241.    Elapsed: 0:03:25.
  Batch   220

<center>[4] 💫</center>

<H3>4. Сохранение</H3>

Давайте теперь вернемся немного назад и вспомним, что мы использовали <i>предобученную</i> модель  
Это значит, что ее уже обучали ранее, получили определенные значения параметров и записали их, а мы их считали и использовали

Нашу модель с новыми параметрами, измененными в процессе дообучения, также можно сохранить  
Тогда в следующий раз мы сможем ее считать и использовать, не обучая заново

In [21]:
import os

# Задаем директорию модели
model_dir = './models/rubert-tiny-trained'
# Если она не существует создаем её
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

print("Saving model to %s" % model_dir)

# Сохраняем натренированную модель и её токенайзер.
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

Saving model to ./models/rubert-tiny-trained


('./models/rubert-tiny-trained\\tokenizer_config.json',
 './models/rubert-tiny-trained\\special_tokens_map.json',
 './models/rubert-tiny-trained\\vocab.txt',
 './models/rubert-tiny-trained\\added_tokens.json')

<center>[5] ⭐⭐⭐</center>

Загрузите заново сохраненную модель, примените ее к датасету CoLA и посчитайте метрику accuracy

In [22]:
import numpy as np
from sklearn.metrics import accuracy_score

model = BertForSequenceClassification.from_pretrained(
    "./models/rubert-tiny-trained",                                 # Используем rubert-tiny.
    num_labels = 2,                                         # Количество выходных слоёв – 2 для бинарной классификации.
    output_attentions = False,                              # Будет ли модель возвращать веса для attention-слоёв. В нашем случае нет.
    output_hidden_states = False,                           # Будет ли модель возвращать состояние всех скрытых слоёв. В нашем случае нет.
)

df = pd.read_csv("./data/cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

sentences = df['sentence'].values
labels = df['label'].values

encoded_dict_rew = tokenizer.batch_encode_plus(
    list(rew),                           # Текст для токенизации.
    add_special_tokens=True,                   # Добавляем '[CLS]' и '[SEP]'
    max_length=64,                             # Дополняем [PAD] или обрезаем текст до 64 токенов.
    padding='max_length',
    truncation=True,
    return_attention_mask=True,                # Возвращаем также attn. masks.
    return_tensors='pt',                       # Возвращаем в виде тензоров pytorch.
)

input_ids = encoded_dict['input_ids']
attention_masks = encoded_dict['attention_mask']
labels = torch.tensor(labels)

dataset_val1 = TensorDataset(input_ids, attention_masks, labels)
validation_dataloader1 = DataLoader(dataset_val1, shuffle=False, batch_size = batch_size)

validation_step(device, model, validation_dataloader1)

Running Validation...
  Batch    10  of    268.    Elapsed: 0:00:04.
  Batch    20  of    268.    Elapsed: 0:00:08.
  Batch    30  of    268.    Elapsed: 0:00:12.
  Batch    40  of    268.    Elapsed: 0:00:15.
  Batch    50  of    268.    Elapsed: 0:00:19.
  Batch    60  of    268.    Elapsed: 0:00:22.
  Batch    70  of    268.    Elapsed: 0:00:26.
  Batch    80  of    268.    Elapsed: 0:00:33.
  Batch    90  of    268.    Elapsed: 0:00:37.
  Batch   100  of    268.    Elapsed: 0:00:42.
  Batch   110  of    268.    Elapsed: 0:00:47.
  Batch   120  of    268.    Elapsed: 0:00:51.
  Batch   130  of    268.    Elapsed: 0:00:56.
  Batch   140  of    268.    Elapsed: 0:01:00.
  Batch   150  of    268.    Elapsed: 0:01:04.
  Batch   160  of    268.    Elapsed: 0:01:08.
  Batch   170  of    268.    Elapsed: 0:01:13.
  Batch   180  of    268.    Elapsed: 0:01:16.
  Batch   190  of    268.    Elapsed: 0:01:20.
  Batch   200  of    268.    Elapsed: 0:01:24.
  Batch   210  of    268.    Elapsed: 

(0.5906370555731788, 0.7039745469083156, '0:01:51')

<center>[5] 💫</center>

<H2>Задание на лабораторную работу</H2>

<b>1. Подготовить для обучения датасет с отзывами на фильмы (./data/train.csv)

Прим.:
- В примере используется таблица с разделителем <code>\t</code>, здесь разделитель <code>,</code>
- Значения <code>label</code> придется преобразовать к числовому виду с помощью <code>LabelEncoder</code> (см. предыдущую лабораторную работу)

<b>2. Обучить rubert-tiny или другую нейронную сеть для обработки естественного языка определять тип отзыва (позитивный или негативный)  
<b>3. Применить обученную нейросеть к тестовым данным (./data/test.csv)

Прим.:
- Для тестовых данных нужно будет создать новый <code>TensorDataset</code> и <code>DataLoader</code> без <code>label</code>
- Для вычисления выходных значений на тестовых данных понадобится новая функция (используйте в качестве примера функцию валидации)

<b>4. Записать результат и отправить боту

Прим.:
- Используйте пример ./data/sample_submission.csv
- Обратите внимание, что модель выдаст для каждого отзыва 2 значения - его принадлежность к классам "позитивный" и "негативный" в отдельности; вам нужно преобразовать их в одно

<b>Максимальная оценка за защиту лабораторной работы зависит от эффективности обучения:
- топ-5 в рейтинге группы и score>0.55 - 10 баллов
- топ-10 в рейтинге группы и score>0.4 - 8 баллов
- score>0.2 - 5 баллов

<i>Прим.: место в рейтинге считается на момент сдачи  
Думайте =)</i>